# Environment Setup

In [1]:
import os
import sys
from collections import Counter

import numpy as np
import open3d as o3d

from tutorials.open3d import visualization

# Python version
assert sys.version_info >= (3, 8)
# Open3D version
assert o3d.__version__ >= "0.17.0"

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
# Point cloud file I/O
filename = "data/input/iScan-Pcd-1-1.ply"
base_name, extension = os.path.splitext(os.path.basename(filename))

output_filepath = "data/output"
if not os.path.exists(output_filepath):
    os.makedirs(output_filepath)

# 1. Point Cloud Filtering

## Step 1. Down-sampling

In [ ]:
def down_sample(input_filename, output_filename):
    # Load the input point cloud file
    pcd = o3d.t.io.read_point_cloud(input_filename)

    # Function uniformly down-samples the point cloud, evenly select 1 point for every k points
    down_sampled_pcd = pcd.uniform_down_sample(every_k_points=10)

    # Save the down-sampled point cloud to an output file
    o3d.t.io.write_point_cloud(output_filename, down_sampled_pcd, write_ascii=False)

In [ ]:
# Perform down sampling
down_sampled_filename = os.path.join(output_filepath, base_name + " - downsampled" + extension)
down_sample(filename, down_sampled_filename)

In [ ]:
# Visualize down sampled result
visualization.visualization(down_sampled_filename)

## Step 2. Crop the point cloud
This step is done in `CloudCompare` software manually.

## Step 3. Outlier removal

In [ ]:
# Input point cloud file
cropped_filename = os.path.join(output_filepath, base_name + " - cropped" + extension)

In [ ]:
def outlier_removal(input_filename, output_filename, visualize=False):
    # Load the input point cloud file
    pcd = o3d.t.io.read_point_cloud(input_filename)

    # Statistical outlier removal
    filtered_pcd, mask = pcd.remove_statistical_outliers(nb_neighbors=20, std_ratio=16.0)

    # Visualize and save the outlier into a separate file for comparison
    if visualize:
        inlier = pcd.select_by_mask(mask)
        outlier = pcd.select_by_mask(mask, invert=True)

        inlier.paint_uniform_color([0.8, 0.8, 0.8])  # color in grey
        outlier.paint_uniform_color([1.0, 0.0, 0.0])  # color in red

        o3d.visualization.draw_geometries([inlier.to_legacy(), outlier.to_legacy()],
                                          window_name="Visualization — {}".format(input_filename),
                                          width=1000, height=800, left=400, top=150)

        # Save the outlier for comparison
        outlier_filename = os.path.join(output_filepath, "outlier.ply")
        o3d.t.io.write_point_cloud(outlier_filename, outlier, write_ascii=False)

    # Save the filtered point cloud to an output file
    o3d.t.io.write_point_cloud(output_filename, filtered_pcd, write_ascii=False)

In [ ]:
# Perform outlier removal
filtered_filename = os.path.join(output_filepath, base_name + " - filtered" + extension)
outlier_removal(cropped_filename, filtered_filename, visualize=True)

# 2. Point Cloud Segmentation

## Step 1. Mask by intensity

In [ ]:
# Input point cloud file
filtered_filename = os.path.join(output_filepath, base_name + " - filtered" + extension)

In [ ]:
def mask_intensity(input_filename, output_filename):
    # Load the input point cloud file
    pcd = o3d.t.io.read_point_cloud(input_filename)
    print(pcd.point, '\n')

    mask = {"Remainder": 0, "Low Intensity": 1, }

    # Mask to acquire low intensity points
    intensity = pcd.point.scalar_intensity.numpy()
    ls = list(map(lambda i: mask["Low Intensity"] if i <= 50 else mask["Remainder"], intensity))
    ls = np.reshape(ls, (len(ls), 1))

    # Save mask as int32, since int64 and bool is not supported for ply
    pcd.point.mask = np.array(ls, dtype=np.int32)
    print(pcd.point)

    # Save the masked point cloud to an output file
    o3d.t.io.write_point_cloud(output_filename, pcd, write_ascii=False)

In [ ]:
# Perform mask by intensity
masked_filename = os.path.join(output_filepath, base_name + " - masked" + extension)
mask_intensity(filtered_filename, masked_filename)

## Step 2. DBSCAN clustering

In [3]:
# Input point cloud file
masked_filename = os.path.join(output_filepath, base_name + " - masked" + extension)

In [4]:
def dbscan_clustering(input_filename, output_filename):
    # Load the input point cloud file
    pcd = o3d.t.io.read_point_cloud(input_filename)

    # Convert form int32 to bool
    mask = pcd.point.mask.numpy()
    mask = [bool(x) for x in mask]

    # Convert mask to index
    index = [i for i, boolean_val in enumerate(mask) if boolean_val]

    # Extract low intensity points, then perform DBSCAN on them
    low_intensity_pcd = pcd.select_by_mask(mask)
    labels = low_intensity_pcd.cluster_dbscan(eps=0.05, min_points=5)
    labels = labels.numpy()

    n_clusters = labels.max() + 1
    print(n_clusters)

    # Two clusters that have the largest number of points
    if n_clusters >= 2:
        counter = Counter(labels)
        most_common = counter.most_common(2)
        numbers, _ = zip(*most_common)

        index_first = [i for i, val in enumerate(labels) if val == numbers[0]]
        index_second = [i for i, val in enumerate(labels) if val == numbers[1]]

        # Add cluster attribute to the input point cloud
        cluster = np.zeros(len(mask))
        pcd.point.cluster = np.reshape(cluster, (len(cluster), 1))

        # Update cluster attribute
        for i in index_first:
            pcd.point.cluster[index[i]] = 1
        for i in index_second:
            pcd.point.cluster[index[i]] = 2

    else:
        print("Warning: DBSCAN clustering return less than 2 clusters.")

    # Save the labeled point cloud to an output file
    o3d.t.io.write_point_cloud(output_filename, pcd, write_ascii=False)

In [5]:
labeled_filename = os.path.join(output_filepath, base_name + " - clustered" + extension)
dbscan_clustering(masked_filename, labeled_filename)

13
